## A conversation with Andrew Ng
이전 CNN에 적용한 데이터셋보다 훨씬 더 크고 복잡한 실세계의 이미지에 적용하는 방법

## Understanding ImageGenerator
* tensorflow의 **image generator** : 한 directory를 지정하면 하위 이미지들에 자동 label을 지정해줌
> * cf. target_size를 지정함으로써 수천 개의 이미지를 전처리할 필요가 없다! (load 와 동시에 resize 하므로)   
> * for 문을 굳이 사용하지 않고 처리 가능.  
   
 "This is coded to read images from subdirectories, and automatically label them from the name of that subdirectory."


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255) # Image Generator 객체 생성, normalize (0~1으로)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (300, 300), # 신경망 학습 시 모두 동일한 크기여야 하므로 이미지 로드 시 조정!
    batch_size = 128, # 하나하나 학습하는 것보다 배치 단위로 불러와서 학습하는 게 더 효율적
    class_mode = 'binary' # 말, 인간 2가지 이진 분류 모드
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size = (300, 300), 
    batch_size = 32, 
    class_mode = 'binary'
)

## Defining a ConvNet to use complex images
* input_shape : (300, 300, 3) **3채널 컬러 이미지**
* output layer : **노드 1개**
> ***이전에는 각 클래스당 1개식 총 10개의 뉴런이엇지만, 지금은 두 클래스에 대해 1개의 뉴런만!***  
* cf. **sigmoid** 활성화 함수 : **이진 분류 (0, 1)**에 적합
* cf. **softmax** 활성화 함수 : 큰 값은 더 크게 활성, 작은 값은 더 작게 억제

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(300, 300, 3)), # ret : (298, 298)
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'), # 64개의 마스크 제공 -> ret : (71, 71)
    tf.keras.layers.MaxPooling(2, 2), # ret : (35, 35) 크기의 이미지, 64개의 컨볼루션(커널, 마스크, 가중치)

    tf.keras.layers.Flatten(), # (35, 35, 64) -> (78400)
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') # 2개의 클래스지만 1개의 출력 노드! (sigmoid)                                      
])

#### cf. Output Shape

* 16, 32, 64... : 커널(컨볼루션 마스크) 개수!
* 35 x 35 x 64 = 78400 : 총 가중치(w) 개수 
> "CNN : **최적의 커널(컨볼루션 마스크 = 가중치)을 자동으로 찾아준다!**"

## Training the Convnet with fit_generator
model.fit 대신 model.fit_generator
* RMSprop (optimizer) : 학습 속도를 조정하여 성능 평가 가능

In [ ]:
from tensorflow.keras.optimizers import RMSprop

# compile : 모델 학습 과정 정의
model.compile(loss='binary_crossentropy', # 이전 loss : categorical cross entropy
              optimizer=RMSprop(lr=0.001), # 이전 optimizer : Adam
              metrics=['acc'])

# fit_generator : 모델 학습
history = model.fit_generator(
    train_generator, # 학습데이터 dir에서 이미지들 불러옴
    steps_per_epoch = 8, # 학습데이터 모두를 로드하려면 batch_size=128 * 8 steps 배치를 수행해야 함!
    epochs = 15, # 학습 epoch 수

    validation_data = validation_generator,
    validation_steps = 8, # batch_size=32 * 8 steps 배치를 수행해야 전체 256개의 val_data 로드
    verbose = 2 # 훈련이 진행되는 동안 표시 할 양 지정 (화살표 2개씩 진행상황 보여줌 * 8 steps)
)

== == == == == == == ==> : 이렇게 == 2개씩 훈련과정 표시하고, 한 == 당 32개의 이미지 데이터들 처리, 총 8 step으로 전체 256개 데이터 다 학습. 이 전체 과정을 총 15 epochs 만큼 반복!

In [ ]:
# 훈련을 마친 모델으로 예측
import numpy as np
from google.colab import files # 불러오기 버튼 창 제공
from keras.preprocessing import image

uploaded = files.upload() # 불러오기 (이미지 경로들이 uploaded 에 불러와짐)
#print(type(uploaded)) : dict?

for fn in uploaded.keys():

  #predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300)) # 위에서 설계한 모델의 input size와 일치하도록 load!
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis = 0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10) # 클래스 배열 반환 : 한 클래스당 한 item(0 or 1에 가까운 값)
  print(classes[0]) # only one value!
  if classes[0] > 0.5:
    print(fn + " is a human") # 1에 가까우면 사람
  else:
    print(fn + " is a horse") # 0에 가까우면 말

## Walking through training the ConvNet with fit_generator



## Adding automatic validation to test accuracy
validation set : 신경망이 이전에 보지 못했던 새로운 데이터이므로, train_acc > val_acc

## Exploring the impact of compressing images
첫번째 Conv2D층의 input shape를 (300, 300) -> (150, 150)으로 줄이면? (데이터 양을 1/4로 축소)
> * **학습 속도**가 빨라짐
* **acc**는 1.000이 나오지만 **val_acc**는 그렇지 않은 값이 나오는 등 **과적합(over-fitting)**이 일어남
* 또한, 작은 크기의 이미지를 다루기 위해 컨볼루션 몇 개를 제거했기 때문에 다른 결과가 도출됨
  
따라서 나중에 test 시 잘못 분류하는 오류가 생김!
  



#### [importance of validation set]
어디에서 오류가 났는지, 어떤 data를 어떻게 수정해야 좋은 학습 결과를 도출해내는지 알 수 있음
> "**validation set**이란, training을 한 후에 만들어진 모형이 잘 예측을 하는지 그 성능을 평가하기 위해서 사용한다. training set의 일부를 모델의 성능을 평가하기 위해서 희생하는 것"